In [1]:
import cobra
from cobra.io.mat import *
import warnings
warnings.simplefilter('ignore')
#from cobra.sampling import sample
from cobra.sampling import OptGPSampler, ACHRSampler
import os
cpus = os.cpu_count()
from cobra.io.json import load_json_model
#model = load_matlab_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.mat")
model = load_matlab_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.mat")

#recon301 =load_matlab_model("./COBRA_models/Recon3DModel_301.mat")

model.objective = 'r0399'

model.reactions.get_by_id('biomass_maintenance').bounds = .1,100
model.reactions.get_by_id('biomass').bounds             = .1, 100
model.reactions.get_by_id('PHETHPTOX2').bounds          = 0, 0

model.reactions.get_by_id("EX_phe_L_e").bounds = (-100, -.1)


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [2]:
model.objective.expression

1.0*r0399 - 1.0*r0399_reverse_b9ea2

In [3]:
model.reactions.get_by_id('PHETHPTOX2').r0399          = 0, 1e10
print(
model.optimize(), #,recon301.optimize(),
model.reactions.get_by_id('PHEyLATthc').bounds,
model.reactions.get_by_id('TYRATB0tc').bounds
)

#len([ r.id for r in model.reactions])- len([ r.id for r in recon301.reactions])

<Solution 100.000 at 0x7f61b0f4efb0> (0.0, 100.0) (0.0, 100.0)


In [4]:
print(
      model.optimize().reduced_costs.loc[lambda x: abs(x)>0].to_string()
      
      
      )

r0399    2.0


In [5]:
print(
      model.optimize().reduced_costs.loc[[r.id for r in model.metabolites.get_by_id("phe_L_c").reactions]].loc[lambda x: abs(x)>1e-14].to_string()
      
      
      )

r0399    2.0


In [6]:
model.reactions.get_by_id("EX_phe_L_e")

Reaction identifier,EX_phe_L_e
Name,Exchange of L-Phenylalanine
Memory address,0x7f619c0fd4e0
Stoichiometry,phe_L_e <-- L-Phenylalanine <--
GPR,
Lower bound,-100
Upper bound,-0.1


In [9]:





def turn_off_rxn(rxn_id):
    model.reactions.get_by_id(rxn_id).bounds       = (0, 0)
    model.reactions.get_by_id("EX_phe_L_e").bounds = (-100, -.1)
    model.reactions.get_by_id("r0399").bounds = (0, 100000.0)
    model.reactions.get_by_id("PHETHPTOX2").bounds = (0, 0.0)
    
    return model.reactions.get_by_id(rxn_id).bounds 
    


list(map(turn_off_rxn, [r.id for r in model.metabolites.get_by_id("tyr_L_e").reactions]))
#list(map(turn_off_rxn, [r.id for r in model.metabolites.get_by_id("tyr_L_c").reactions]))

#list(map(turn_off_rxn, [r.id for r in model.metabolites.get_by_id("phe_L_e").reactions]))

list(map(turn_off_rxn, [r.id for r in model.metabolites.get_by_id("dhbpt_e").reactions]))
list(map(turn_off_rxn, [r.id for r in model.metabolites.get_by_id("thbpt_e").reactions]))
#list(map(turn_off_rxn, [r.id for r in model.metabolites.get_by_id("thbpt4acam_e").reactions]))


print(
model.optimize(), #,recon301.optimize(),
model.reactions.get_by_id('PHEyLATthc').bounds,
model.reactions.get_by_id('TYRATB0tc').bounds,
model.reactions.get_by_id("EX_phe_L_e").bounds 
)





<Solution 300.000 at 0x7f619dfe30a0> (0.0, 100.0) (0, 0) (-100, -0.1)


In [39]:
print(
      model.optimize().reduced_costs.loc[lambda x: abs(x)>1e-14].to_string()
      
      
      )




regulatory_reactions = model.optimize().reduced_costs.loc[lambda x: abs(x)>1e-14].index.tolist()



[model.reactions.get_by_id(rr).bounds for rr in regulatory_reactions]

            
   
                 
             
            
         


model.reactions.get_by_id("DHPR2").bounds = (0, 10)
model.reactions.get_by_id("THBPT4ACAMDASE").bounds = (0, 10)
model.reactions.get_by_id("r0403").bounds = (0, 10)
model.reactions.get_by_id("DHPR").bounds = (0, 10)
model.reactions.get_by_id("r0398").bounds = (0, 10)
model.reactions.get_by_id("HMR_6728").bounds = (0, 10)


model.optimize()

DHPR2             2.0
THBPT4ACAMDASE   -2.0
r0403            -2.0
EX_dhbpt_e        2.0
DHPR              2.0
r0398             2.0
HMR_6728         -2.0


,fluxes,reduced_costs
10FTHF5GLUtl,0.000000,0.0
10FTHF5GLUtm,0.000000,0.0
10FTHF6GLUtl,0.000000,0.0
10FTHF6GLUtm,0.000000,0.0
10FTHF7GLUtl,0.000000,0.0
...,...,...
NADH2_u10mi,0.000000,0.0
CYOOm3i,0.000000,0.0
CYOOm2i,54.335796,0.0
ARTPLM1,1.000000,0.0


In [11]:


from cobra.flux_analysis import flux_variability_analysis


print( model.reactions.get_by_id("EX_phe_L_e").bounds )
FVA =  flux_variability_analysis(model, fraction_of_optimum=.95, processes=os.cpu_count())



(-100, -0.1)


In [15]:
import pandas as pd


fva = FVA.loc[ [r.id for r in model.metabolites.get_by_id("tyr_L_c").reactions]]



print(fva.to_string())

                     minimum     maximum
TYRLEUARGr            -100.0  100.000000
r1829                    0.0    0.000000
TYRPHETYRr            -100.0  100.000000
r1788                    0.0    0.000000
r1588                    0.0    0.000000
r1859                    0.0    0.000000
r1950                    0.0    0.000000
TYRTHRr               -100.0  100.000000
r1999                    0.0    0.000000
TYRTRPPHEr            -100.0  100.000000
r1789                    0.0    0.000000
r1889                    0.0    0.000000
r1777                    0.0    0.000000
TYRTYRr               -100.0  100.000000
r1951                    0.0    0.000000
TYRTA                 -100.0  100.000000
LEUTYRTYRr            -100.0  100.000000
r1753                    0.0    0.000000
TYRVALMETr            -100.0  100.000000
r1627                    0.0    0.000000
ASNTYRGLYr            -100.0  100.000000
r2063                    0.0    0.000000
r1790                    0.0    0.000000
r1678           

In [ ]:
rxns        = [ r.id for r in model.reactions]





model.reactions.get_by_id("r0399").bounds       = (0, 1e-3)
model.reactions.get_by_id("PHETHPTOX2").bounds  = (0, 1e-3)



In [10]:


model.reactions.get_by_id("r0403").bounds 
model.reactions.get_by_id("r0402").bounds 
model.reactions.get_by_id("TRPHYDRO2").bounds 
model.reactions.get_by_id("THBPT4ACAMDASE").bounds 


model.reactions.get_by_id("RE1709C").bounds 

(-10, 10)

In [16]:
import re
from itertools import compress


rxns        = [ r.id for r in model.reactions]


subsystems  = [s.subsystem for s in model.reactions]
subsys_bool = [bool(re.search("Phen|phen|Tetrahydrobiopterin", s)) for s in subsystems]
subsys_rxns = list(compress(rxns, subsys_bool))

reversible = [
'PHETA1',
'PHETA1m',
'r0547',
'RE0830C',
'RE0830N',
'RE1709C',
'RE1709N',
'RE2660C',
'RE2660N']

forward_rxns =  list(set(subsys_rxns)-set(reversible))

for r in forward_rxns:
    model.reactions.get_by_id(r).bounds  = (1e-8, 10)
    
    
for r in reversible:
    model.reactions.get_by_id(r).bounds  = (-10, 10)

co    = model.problem.Constraint(model.reactions.biomass.flux_expression,
                                 lb=.90 * model.optimize().objective_value,
                                 ub=.999 * model.optimize().objective_value)
model.add_cons_vars([co])
assert model.optimize().objective_value > 0
print(
model.optimize(), #,recon301.optimize(),
cpus)



model.reactions.get_by_id("biomass").bounds  

<Solution 31.042 at 0x7f45fe814460> 20


(0.0, 100.0)

In [20]:
model.reactions.get_by_id("biomass").bounds = .90 * model.optimize().objective_value, model.optimize().objective_value
model.reactions.get_by_id("biomass").bounds

(27.937686205219254, 31.041873561354727)

In [3]:
model.reactions.get_by_id("r0399").bounds       = (0, 1e-3)
model.reactions.get_by_id("PHETHPTOX2").bounds  = (0, 1e-3)


assert model.optimize().objective_value > 0
model.optimize().objective_value, model.optimize().to_frame().loc["r0399"], model.optimize().to_frame().loc["PHETHPTOX2"]




(30.793010052375,
 fluxes           0.0
 reduced_costs    0.0
 Name: r0399, dtype: float64,
 fluxes           0.0
 reduced_costs    0.0
 Name: PHETHPTOX2, dtype: float64)

In [4]:
optgp = OptGPSampler(model, processes=cpus, thinning=100)
samples = optgp.sample(7_000)
samples.to_parquet("./results/fluxes/flux_samples_PKU_7_000.parquet.gzip", compression='gzip')  

import pandas as pd
flux_samples_PKU_7_000 = pd.read_parquet("./results/fluxes/flux_samples_PKU_7_000.parquet.gzip")  
flux_samples_PKU_7_000[["PHETHPTOX2", "r0399"]]

,PHETHPTOX2,r0399
0,0.000011,0.000011
1,0.000011,0.000011
2,0.000011,0.000011
3,0.000011,0.000011
4,0.000011,0.000011
...,...,...
6995,0.000480,0.000022
6996,0.000404,0.000065
6997,0.000391,0.000065
6998,0.000388,0.000062


In [5]:
model.reactions.get_by_id("r0399").bounds       = (30, 100)
model.reactions.get_by_id("PHETHPTOX2").bounds  = (30, 100)
assert model.optimize().objective_value > 0
model.optimize().to_frame().loc["r0399"], model.optimize().to_frame().loc["PHETHPTOX2"]
model.optimize()

,fluxes,reduced_costs
10FTHF5GLUtl,0.000000,0.0
10FTHF5GLUtm,0.000000,0.0
10FTHF6GLUtl,0.000000,0.0
10FTHF6GLUtm,0.000000,0.0
10FTHF7GLUtl,0.000000,0.0
...,...,...
NADH2_u10mi,0.000000,0.0
CYOOm3i,0.000000,0.0
CYOOm2i,41.666667,0.0
ARTPLM1,0.000000,0.0


In [6]:

optgp = OptGPSampler(model, processes=cpus, thinning=100)
samples = optgp.sample(7_000)
samples.to_parquet("./results/fluxes/flux_samples_CONTROL_7_000.parquet.gzip", compression='gzip')  
import pandas as pd
flux_samples_CONTROL_7_000 = pd.read_parquet("./results/fluxes/flux_samples_CONTROL_7_000.parquet.gzip")  
flux_samples_CONTROL_7_000[["PHETHPTOX2", "r0399"]]

,PHETHPTOX2,r0399
0,30.0,30.038785
1,30.0,30.039390
2,30.0,30.037066
3,30.0,30.039025
4,30.0,30.037152
...,...,...
6995,30.0,40.786187
6996,30.0,40.774335
6997,30.0,40.745141
6998,30.0,40.782505


optgp = OptGPSampler(model, processes=cpus, thinning=1000)
samples = optgp.sample(10_000)
samples.to_parquet("./results/fluxes/flux_samples.parquet.gzip", compression='gzip')  

### Get non zero samples from phenylalanine and biopterin subsystems

In [1]:
import pandas as pd
from cobra.io.mat import *
from itertools import compress

model                      = load_matlab_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.mat")
flux_samples_CONTROL_7_000 = pd.read_parquet("./results/fluxes/flux_samples_CONTROL_7_000.parquet.gzip")  
flux_samples_PKU_7_000     = pd.read_parquet("./results/fluxes/flux_samples_PKU_7_000.parquet.gzip")  


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [3]:


rxns        = [ r.id for r in model.reactions]
subsystems  = [s.subsystem for s in model.reactions]
subsys_bool = [bool(re.search("Phen|phen|Tetrahydrobiopterin", s)) for s in subsystems]
pd.Series(subsystems).iloc[subsys_bool].unique()


array(['Tetrahydrobiopterin metabolism', 'Phenylalanine metabolism'],
      dtype=object)

In [4]:
subsys_rxns = list(compress(rxns, subsys_bool))
subsys_rxns

['DHPR2',
 'PACCOAL',
 'PEAMNO',
 'PHACCOAGLNAC',
 'PHETA1m',
 'PHETHPTOX2',
 'PHYCBOXL',
 'PPOR',
 'PTHPS',
 'SPR',
 'THBPT4ACAMDASE',
 'r0399',
 'r0402',
 'r0403',
 'r0545',
 'r0547',
 'r0673',
 'RE0830C',
 'RE0830N',
 'RE1709C',
 'RE1709N',
 'RE2660C',
 'RE2660N',
 'PHLAC',
 '3HPPPNOHc',
 '3HPPPNOHGLUCc',
 'GLYNATm',
 'PACCOALm',
 'DHPR',
 'GTPCI',
 'r0398',
 'PHETA1',
 'HMR_6728',
 'HMR_6729',
 'HMR_6747',
 'HMR_6755',
 'HMR_6770',
 'HMR_6782',
 'HMR_6784',
 'HMR_6785',
 'HMR_6786',
 'HMR_6790',
 'HMR_6826',
 'HMR_6834',
 'HMR_6838',
 'HMR_6839',
 'HMR_6844',
 'HMR_6850',
 'HMR_6854',
 'HMR_6874',
 'HMR_6876',
 'HMR_7628',
 'HMR_7756',
 'HMR_4816']

In [13]:

def samples_subsys_rxns(samples, subsys_rxns):

    samples_subsys_rxns = samples[subsys_rxns]

    return samples_subsys_rxns.loc[:,
            list(samples_subsys_rxns.sum().abs() > 1e-10 )]


samples_subsys_CONTROL_7_000 = samples_subsys_rxns(flux_samples_CONTROL_7_000, subsys_rxns)
samples_subsys_PKU_7_000 = samples_subsys_rxns(flux_samples_PKU_7_000, subsys_rxns)
all(samples_subsys_CONTROL_7_000.columns == samples_subsys_PKU_7_000.columns)

#samples_subsys_rxns.loc[:,
#list(samples_subsys_rxns.sum().abs() > 1e-7 )].to_parquet("./results/fluxes/subsys_fluxes_non_zero.parquet.gzip")


True

In [15]:
samples_subsys_CONTROL_7_000.loc[:,
list(samples_subsys_CONTROL_7_000.sum().abs() > 1e-7 )].to_parquet("./results/fluxes/samples_subsys_CONTROL_7_000.parquet.gzip")
samples_subsys_PKU_7_000.loc[:,
list(samples_subsys_PKU_7_000.sum().abs() > 1e-7 )].to_parquet("./results/fluxes/samples_subsys_PKU_7_000.parquet.gzip")